# Actors' operation

<div class="admonition note">
    <p class="admonition-title">In progress</p>
    <p>This document is a work in progress if you see any errors, or exclusions or have any problems, please <a href="https://github.com/absespy/ABSESpy/issues">get in touch with us</a>.</p>
</div>

## Manipulate an individual `Actor`

In [1]:
from abses import Actor, MainModel

# create a testing model
model = MainModel(name="actor_example", base="tests")
layer = model.nature.create_module(how="from_resolution", shape=(3, 3))

# actor creation should be bounded with a model.
actor = model.agents.new(Actor, singleton=True)

# actor has a class-property, -its breed, class's name.
actor.breed  # same as `actor.__class__.__name__`

[2024-03-20 18:35:51][human          ] | Initializing a new Human Module...
[2024-03-20 18:35:51][nature         ] | Initializing a new Base Nature module...
[2024-03-20 18:35:51][nature         ] | Initializing a new Model Layer...
[2024-03-20 18:35:51][nature         ] | Using rioxarray version: 0.13.4
[2024-03-20 18:35:51][container      ] | Created 1 actors of breed Actor


'Actor'

In [2]:
# In an initial condition, this actor is not on the earth...
actor.on_earth

False

## Manipulate a group of `Actors`
`ABSESpy` provides two different container of actors:
- `AgentsContainer`: A unique `dictionary`-like class where all agents of the model are saving.
- `ActorsList`: A `list`-like class where referring some actors temporally.

### AgentsContainer



When the model is created, there is a unique `AgentsContainer` obj bounding. It saves agents by their breeds. In the tutorial above, only one actor is added now.

In [3]:
repr(model.agents)

'<ModelAgents: (1)Actor>'

Container makes creating actors much easier:

In [4]:
# define a new breed of actor
class Seller(Actor):
    name = "seller"


class User(Actor):
    name = "user"


# You'd better to assign the `singleton=True` when only creating one actor.
# Otherwise you would get a list of actors (length = 1).
another_actor = model.agents.new(Actor, singleton=True)
seller = model.agents.new(Seller)  # default creating one actor.
users = model.agents.new(User, 5)  # creating 5 actors

model.agents

[2024-03-20 18:35:51][container      ] | Created 1 actors of breed Actor
[2024-03-20 18:35:51][container      ] | Created 1 actors of breed Seller
[2024-03-20 18:35:51][container      ] | Created 5 actors of breed User


<ModelAgents: (2)Actor; (1)Seller; (5)User>

While `AgentsContainer` mainly provides a way to store, most of manipulations for actors are implemented in another data type: `ActorsList`. `AgentsContainer` also has many ways to convert existing agents to this data type:

In [5]:
# select specific breeds.
model.agents.get(breeds="Seller")

<ActorsList: (1)Seller>

In [6]:
model.agents.get(breeds=["Seller", "User"])

<ActorsList: (1)Seller; (5)User>

In [7]:
# also accessible through selecting query syntax.
model.agents.select("name == user")

<ActorsList: >

For more info about the query syntax, please refer [API docs](../../api/agents.md).

### `ActorsList`

`ActorsList` is a collection of `Actors`, facilitate to manipulate them at a batch.

In [8]:
lst = model.agents.get()
actor = lst[0]  # indexing -> an `Actor`
five_actors = lst[:5]  # slice -> another `ActorsList` object

five_actors  # (2)Actor; (1)Seller; (2)User
# five_actors contains this actor
actor in five_actors

True

Convert the list to a dictionary: `{breed: ActorsList}`

In [9]:
# sort up mixed actors.
five_actors.to_dict()

{'Actor': <ActorsList: (2)Actor>,
 'Seller': <ActorsList: (1)Seller>,
 'User': <ActorsList: (2)User>}

Select by conditions.

In [10]:
five_actors.select(selection="User")  # same as .to_dict()['User']

# select the first and the fourth actors
five_actors.select(selection=[True, False, False, True, False])

<ActorsList: (1)Actor; (1)User>

Actors' attributes are accessible as `np.ndarray`.

In [11]:
# get each Actor(object)'s unique id.
five_actors.array("unique_id")

array([2, 1, 3, 7, 6])

You can access all actors in this list with `id >= 7` by:

In [12]:
better_guys = five_actors.better(metric="unique_id", than=3)
better_guys

<ActorsList: (2)User>

Split the list in the second, third places: `[1, 2, | 3, | 4, 5]`

In [13]:
five_actors.split([2, 3])

[array([<Actor [2]>, <Actor [1]>], dtype=object),
 array([<Seller [3]>], dtype=object),
 array([<User [7]>, <User [6]>], dtype=object)]